<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/moded_spectrum_analysis_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectrum Analysis V2

# Loading Necesary Libraries

In [1]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
from tqdm import tqdm

from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

# Load Enviroment

In [2]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Load Datasets

In [4]:
input_path = path + "/Spectrum/16QAM/25spans80km_withoutROADMs/dataSet_Spectrum31MHz_Samples_16QAM_75GHz_LongHaul_input_25x80km.xlsx"

## Pre-Processing

In [5]:
df = pd.read_excel(input_path, sheet_name = "Sheet1", skiprows=1)

In [6]:
# Dropping rows that are empty
drop_rows = [drop_row for drop_row in range(325, df.shape[0])]
df_13_samples = df.drop(drop_rows)

In [7]:
print([row for row in range(1, 14)])
print([row for row in range(0, 25)])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [8]:
# Adding missing PBRS_id
snippet = []
val_cnt = 1
idx = 0
for sample in range(1, 14):
  for sample_id in range(0, 25):
    df_13_samples.at[idx, 'PBRS_id'] = val_cnt
    idx +=1
    snippet.append(val_cnt)
  val_cnt += 1
df_13_samples['PBRS_id'] = df_13_samples['PBRS_id'].astype(int)

In [9]:

df_13_samples.iloc[0].fillna(method='bfill', inplace=True)

df_13_samples.iloc[1:df_13_samples.shape[0]].fillna(method='pad', inplace=True)

# Cut Tails with on wanted data

## Calculate the average value of groups

In [11]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
      break
    i+=1
  return prime_nums

def aggreagator_v2(df, stop_gap=50):
  total_num_of_data = df.shape[1]
  arr_prime = prime_number_finder(stop_gap, total_num_of_data)
  #agg_num = np.max(arr_prime)
  agg_num = arr_prime[0]

  # Creating new df of aggregate values
  agg_df = pd.DataFrame()

  mid_point = df.shape[1]/2

  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    # Obtain current last columns stop
    agg=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg]]

    init_column_cnt += (agg_num)

    headers = list(new_df.columns.values) 
    if loop_cnt <= mid_point:
      # Get average of freq values for new header
      new_header_name  = headers[-1]
    else:
      new_header_name = headers[0]
    
    # Add aggragated values to new df
    agg_df[new_header_name] = new_df.mean(axis = 1)
  return agg_df

## Select the first sample for testing of the mean values

## Find cut off index

In [12]:
def remove_tail(df, cut_val, sample_id):
  columns_selected = []
  old_val = 0
  delta = 0
  mid_point = int(df.shape[1]/2)
  cols_headers  = list(df.columns.values)

  for i in range(0, df.shape[1]):
    if i == 0:
      cell_val = df.iloc[[0], i]
      old_val = cell_val[sample_id]
    else:
      cell_val = df.iloc[[0], i]
      val = cell_val[sample_id]
      delta = abs(old_val-val)
      old_val = val
      if delta > cut_val:
        if i <= mid_point:
          col_name_selected = cols_headers[i+2]
        else:
          col_name_selected = cols_headers[i-2]
          col_name_selected = round(float(col_name_selected), 5)
        columns_selected.append(col_name_selected)

  return columns_selected

In [13]:
# Removing tails conducting multiples loops with variable cutoff value until we get only two columns
def remove_tail_main(df, cut_val, sample_id):
  old_drop_cols = []
  drop_cols = remove_tail(df, cut_val, sample_id)
  while len(drop_cols) != 2:  
    # If len of columns to be dropped are 0 the select the 2 values smaller and bigger of the previous iteration
    if len(drop_cols) == 0 or len(drop_cols) == 1:
      drop_cols = [np.min(old_drop_cols), np.max(old_drop_cols)]
      break
    else:
      old_drop_cols = drop_cols
      drop_cols = remove_tail(df, cut_val, sample_id)
      cut_val += 2
  return drop_cols

In [14]:
def get_working_data_idx_v2(df_mean_sample, sample_id, cut_val = 0):

  df = df_mean_sample.iloc[[sample_id - 1]]

  agg_df = aggreagator_v2(df)

  col_vals = remove_tail_main(agg_df, cut_val=cut_val, sample_id=sample_id)

  left_index_no = df.columns.get_loc(col_vals[0])
  rigth_index_no = df.columns.get_loc(col_vals[1])

  return left_index_no, rigth_index_no

In [15]:
def cur_off_idx(df):
  df_mean_sample = df.groupby(['PBRS_id']).mean()
  df_mean_sample = df_mean_sample.drop(['row', 'Channels', 'Distance_km', 'power_dBm'], axis = 1)

  cut_points = {}
  print('Calculating the cutoff values:')
  for sample in tqdm(df_mean_sample.index):
    left_index_no, rigth_index_no = get_working_data_idx_v2(df_mean_sample, sample_id = sample)
    cut_points[sample] = (left_index_no, rigth_index_no)

  left_cut_off = np.max([v[0] for k, v in cut_points.items()])
  rigth_cut_off = np.min([v[1] for k, v in cut_points.items()])

  cut_point = (left_cut_off, rigth_cut_off)
  return cut_point

In [16]:
cut_point = cur_off_idx(df_13_samples)

Calculating the cutoff values:


100%|██████████| 13/13 [00:10<00:00,  1.20it/s]


## Select working data

In [17]:
def select_working_data(df, cut_point):
  extra_info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, cut_point[0]+6:cut_point[1]+6]
  smoothed_data_frame = pd.DataFrame(savgol_filter(data_df, window_length = 5, polyorder = 2))
  
  # Adding back headers to the smoothed data
  rename_col = {}
  cnt = 0
  for col in data_df.columns:
    h_col = round(float(col), 5)
    rename_col[cnt] = h_col
    cnt += 1
  smoothed_data_frame.rename(columns=rename_col, inplace=True)

  working_df = pd.merge(extra_info_df, smoothed_data_frame, left_index=True, right_index=True)
  return working_df

In [35]:
working_df = select_working_data(df_13_samples, cut_point)
working_df

,row,PBRS_id,Channels,Distance_km,power_dBm,#span,-32.15625,-32.125,-32.09375,-32.0625,...,32.125,32.15625,32.1875,32.21875,32.25,32.28125,32.3125,32.34375,32.375,32.40625
0,1,1,1,80.0,0,1,-14.851402,-21.638525,-23.529105,-14.969324,...,-19.957216,-19.447732,-14.576655,-15.491358,-14.654316,-17.801658,-16.387551,-13.980891,-15.122689,-17.753853
1,2,1,1,160.0,0,2,-32.415520,-23.442737,-17.642596,-16.382163,...,-15.348137,-13.125596,-15.424900,-18.560355,-17.197693,-12.466164,-12.735882,-10.007690,-13.820587,-21.908636
2,3,1,1,240.0,0,3,-23.428437,-21.738735,-19.956032,-18.320372,...,-21.210045,-20.014100,-13.636612,-9.728106,-7.977970,-19.904029,-22.658427,-19.765777,-24.603827,-31.921519
3,4,1,1,320.0,0,4,-14.298403,-15.210905,-15.951684,-16.209202,...,-18.673789,-12.470911,-12.638057,-9.282286,-12.488977,-10.933173,-15.749634,-15.676254,-15.412170,-14.347517
4,5,1,1,400.0,0,5,-19.455200,-19.755724,-18.313774,-13.272877,...,-17.697814,-16.074418,-16.232349,-13.788018,-22.471867,-23.016590,-22.392253,-18.544159,-18.765642,-23.196090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,321,13,1,1680.0,0,21,-31.466739,-32.655263,-31.882492,-27.589879,...,-34.359245,-32.466935,-29.543434,-31.321699,-33.322457,-34.343455,-30.096108,-33.185712,-33.367392,-33.876192
321,322,13,1,1760.0,0,22,-21.303913,-33.080588,-37.034437,-31.766035,...,-32.977742,-28.624796,-27.495186,-23.847968,-25.561252,-34.242901,-40.735728,-34.014182,-33.957388,-35.627786
322,323,13,1,1840.0,0,23,-27.601906,-27.758137,-27.102083,-27.319948,...,-33.646975,-25.620092,-34.883694,-42.119980,-35.870338,-28.109018,-32.237494,-32.837088,-31.096978,-25.976283
323,324,13,1,1920.0,0,24,-31.725625,-38.937830,-39.113483,-26.640446,...,-21.790284,-24.651905,-26.129703,-26.548903,-25.567147,-30.324506,-30.829273,-33.027456,-34.010274,-34.402337


In [19]:
df_mean_sample = df_13_samples.groupby(['PBRS_id']).mean()
df_mean_sample = df_mean_sample.drop(['row', 'Channels', 'Distance_km', 'power_dBm'], axis = 1)

In [24]:
'''
test_df = df_mean_sample.iloc[[0]]
sample_1 = test_df.iloc[0].squeeze()
sample_1.plot()
plt.show()
'''

'\ntest_df = df_mean_sample.iloc[[0]]\nsample_1 = test_df.iloc[0].squeeze()\nsample_1.plot()\nplt.show()\n'

In [25]:
'''
smoothed_2dg = savgol_filter(test_df, window_length = 5, polyorder = 2)
gfg = pd.Series(smoothed_2dg[0])
gfg.plot()
plt.show()
'''

'\nsmoothed_2dg = savgol_filter(test_df, window_length = 5, polyorder = 2)\ngfg = pd.Series(smoothed_2dg[0])\ngfg.plot()\nplt.show()\n'

# Pearson Correlation

In [88]:
#eval_method = 'avg' or 'min' or 'max'

def pearson_matrix(working_df , source_sample_id, eval_method = 'avg'):
  distances = [x*80 for x in range(1, 26)]
  result_data = {'source/target': distances}
  print('Generating Pearson Confusion Matrix using ({}) method:'.format(eval_method))
  for target_dist in tqdm(distances): # loop all distances available
    result_target = []
    for source_dist in distances: # loop all distances available
      source_df = working_df[(working_df['Distance_km'] == source_dist) & (working_df['PBRS_id'] == source_sample_id)]# select data of source (distance and sample_id) 
      source_data = source_df.iloc[0, 6:source_df.shape[1]].tolist()
      pearson_values = []
      for target_sample_id in working_df['PBRS_id'].unique(): # loop all samples id to then exclude the source sample id
        if target_sample_id != source_sample_id:
          target_df = working_df[(working_df['Distance_km'] == target_dist) & (working_df['PBRS_id'] == target_sample_id)] # select data of target
          target_data = target_df.iloc[0, 6:target_df.shape[1]].tolist()

          pearson_values.append(round(pearsonr(source_data, target_data)[0], 5)) # pearson correlation
      if eval_method == 'avg':
        p_result = np.mean(pearson_values)
      elif eval_method == 'max':
        p_result = np.max(pearson_values)
      elif eval_method == 'min':
        p_result = np.min(pearson_values)
      result_target.append(p_result)
    result_data[target_dist] = result_target

  pearson_matrix_df = pd.DataFrame.from_dict(result_data)
  #pearson_matrix_df = pearson_matrix_df.set_index('source/target')

  return pearson_matrix_df, result_data

In [89]:
avg_p_matrix, avg_result_data = pearson_matrix(working_df , source_sample_id=1, eval_method = 'avg')
min_p_matrix, min_result_data = pearson_matrix(working_df , source_sample_id=1, eval_method = 'min')
max_p_matrix, max_result_data = pearson_matrix(working_df , source_sample_id=1, eval_method = 'max')

Generating Pearson Confusion Matrix using (avg) method:


100%|██████████| 25/25 [00:22<00:00,  1.10it/s]


Generating Pearson Confusion Matrix using (min) method:


100%|██████████| 25/25 [00:23<00:00,  1.08it/s]


Generating Pearson Confusion Matrix using (max) method:


100%|██████████| 25/25 [00:23<00:00,  1.08it/s]


In [90]:
avg_p_matrix_output_path = path + '/Spectrum/confusion_matrix_pearson/Correlation_Pearson_AVG.csv'
avg_p_matrix.to_csv(avg_p_matrix_output_path, index=False)  

min_p_matrix_output_path = path + '/Spectrum/confusion_matrix_pearson/Correlation_Pearson_MIN.csv'
min_p_matrix.to_csv(min_p_matrix_output_path, index=False)  

max_p_matrix_output_path = path + '/Spectrum/confusion_matrix_pearson/Correlation_Pearson_MAX.csv'
max_p_matrix.to_csv(max_p_matrix_output_path, index=False)  

## Pickle out the dictionary of correlations

In [91]:
def pickle_out(dump_file, filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  pickleout = open(filepath, "wb")
  pickle.dump(dump_file, pickleout)
  pickleout.close()

In [92]:
pickle_out(avg_result_data, 'avg_correlation_matrix')
pickle_out(min_result_data, 'min_correlation_matrix')
pickle_out(max_result_data, 'max_correlation_matrix')